In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
import os
import pandas as pd

from keras.models import Model
from keras.layers import Input, Dense, Flatten, Reshape, Layer
from keras.datasets import mnist
from keras.utils import to_categorical

import keras.backend as K

# Data

In [3]:
def preprocess(data):
    x, y = data
    x = x.reshape((len(x), 28, 28))
    x = x/255.
    #y = to_categorical(y, 10) #don't need to categorise y
    return x, y

train, test = mnist.load_data()

x_train, y_train = preprocess(train)
x_test, y_test = preprocess(test)

print(x_train.shape)

(60000, 28, 28)


# Model

In [4]:
# VAE model from "Auto Encoding Variation Bayes": https://arxiv.org/pdf/1312.6114.pdf
# encoder model is modelled by a gaussian, producing the latent
# variables mean and its log standard deviation


def encoder_model(latent_dim, hidden_dim=512):
    enc_in = Input(shape=(28,28), name='enc_input')
    enc_flat = Flatten(name='enc_flatten')(enc_in)
    enc_hid = Dense(hidden_dim, activation='relu', name='enc_hidden')(enc_flat)
    z_mean = Dense(latent_dim, name='latent_mean')(enc_hid)
    log_z_var = Dense(latent_dim, name='log_latent_var')(enc_hid)
    enc_model = Model(enc_in, [z_mean, log_z_var])
    return enc_model, (z_mean, log_z_var)

# from the paper, we can either model the decoder p(x|z) as a bernoulli distribution, 
# or as a gaussian distribution, we'll try both

def bernoulli_decoder(latent_dim, hidden_dim=512):
    dec_in = Input(shape=(latent_dim,), name='dec_in')
    dec_hid = Dense(hidden_dim, activation='relu', name='dec_hidden')(dec_in)
    dec_flat = Dense(28*28, activation='sigmoid', name='dec_flat')(dec_hid)
    dec_out = Reshape((28, 28), name='dec_out')(dec_flat)
    dec_model = Model(dec_in, dec_out)
    return dec_model

def gauss_decoder(latent_dim, hidden_dim=512):
    dec_in = Input(shape=(latent_dim,), name='dec_in')
    dec_hid = Dense(hidden_dim, activation='relu', name='dec_hidden')(dec_in)
    dec_mean = Dense(28*28, name='dec_mean')(dec_hid)
    log_dec_var = Dense(28*28, name='log_dec_var')(dec_hid)
    dec_model = Model(dec_in, [dec_mean, log_dec_var])
    return dec_model

# we need a layer that samples a latent variable given a mean and standard deviation

class Sampler(Layer):
    
    def __init__(self, stddev, **kwargs):
        self.stddev = stddev
        super(Sampler, self).__init__(**kwargs)
    
    def call(self, x, training=None):
        assert isinstance(x, list)
        z_mean, log_z_var = x
        z_std = K.exp(log_z_var/2)
        # sample epsilon from N(0, stddev)
        shape = K.shape(z_std)
        epsilon = K.random_normal(shape, mean=0, stddev=self.stddev)
        # we sample during training only
        z_sample = z_mean + z_std * epsilon
        return K.in_train_phase(z_sample, z_mean, training=training)
    
    def compute_output_shape(self, input_shape):
        assert isinstance(input_shape, list)
        assert input_shape[0] == input_shape[1]
        return input_shape[0]

# Bernoulli Model

In [84]:
latent_dim = 200
hidden_dim = 512

enc_model, (z_mean, log_z_var) = encoder_model(latent_dim, hidden_dim)
dec_model = bernoulli_decoder(latent_dim, hidden_dim)

# loss
def bernoulli_loss(y_true, y_pred):
    kl_loss = - 1/2 * K.sum(1 + log_z_var - K.square(z_mean) - K.exp(log_z_var), axis=-1)
    im_loss = K.sum(K.binary_crossentropy(y_true, y_pred), axis=(-1, -2))
    return kl_loss + im_loss

In [85]:
# make the z_sampler layer to connect the decoder and encoder
z_sampler = Sampler(1, name='z_sampler')([z_mean, log_z_var])
# connect the encoder and decoder
enc_dec_model = Model(enc_model.input, dec_model(z_sampler))

# train the model
epochs = 30
batch_size = 256

enc_dec_model.compile(optimizer='adam', loss=bernoulli_loss)

enc_dec_model.fit(
    x=x_train, y=x_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(x_test, x_test)
)

Train on 60000 samples, validate on 10000 samples
Epoch 1/30
60000/60000 [==============================] - 7s 120us/step - loss: 190.6364 - val_loss: 139.9026
Epoch 2/30
60000/60000 [==============================] - 6s 94us/step - loss: 144.6420 - val_loss: 123.0458
Epoch 3/30
60000/60000 [==============================] - 6s 93us/step - loss: 134.5733 - val_loss: 116.0362
Epoch 4/30
60000/60000 [==============================] - 6s 94us/step - loss: 126.7796 - val_loss: 109.5051
Epoch 5/30
60000/60000 [==============================] - 6s 93us/step - loss: 120.7817 - val_loss: 106.0872
Epoch 6/30
60000/60000 [==============================] - 6s 94us/step - loss: 116.9149 - val_loss: 103.6066
Epoch 7/30
60000/60000 [==============================] - 6s 101us/step - loss: 114.1128 - val_loss: 102.2472
Epoch 8/30
60000/60000 [==============================] - 9s 154us/step - loss: 112.1076 - val_loss: 99.7250
Epoch 9/30
60000/60000 [==============================] - 9s 153us/step - lo

# Visualising

In [86]:
# first make a table of elbo values
def elbo(i, x):
    # keras doesn't have a good way of storing the losses for individual images, so we'll use this for now
    return enc_dec_model.evaluate(x[i:i+1], x[i:i+1], verbose=0)

file_path = './mnist_elbos_%d_dim.csv' %latent_dim

if not os.path.exists(file_path):
    elbos = [-elbo(i, x_train) for i in range(len(x_train))]
    df_elbos = pd.DataFrame(elbos, columns=['elbo'])
    df_elbos.to_csv(file_path)
else:
    df_elbos = pd.read_csv(file_path, index_col=0)
    
y_df = pd.DataFrame(y_train, columns=['label'])

In [91]:
# make a table of the min, max, mean, and std of the elbos for the various numerical labels in our dataset
elbo_labels = []

for num in range(10):
    idx = y_df[y_df['label'] == num].index
    df_nums = df_elbos.iloc[idx, 0]
    elbo_min = df_nums.min()
    elbo_max = df_nums.max()
    elbo_mean = mean(df_nums)
    elbo_std = std(df_nums)
    elbo_labels.append([elbo_min, elbo_max, elbo_mean, elbo_std])

# indices represent the number
df_elbo_labels = pd.DataFrame(elbo_labels, columns=['Min', 'Max', 'Mean', 'Std'])

# save the data
elbo_path = './mnist_labels_elbo_%d_dim.xlsx' %latent_dim
df_elbo_labels.to_excel(elbo_path, index_label='Number')

In [92]:
# order the data by the mean, find the numeral with the highest average elbo
df_elbo_labels.sort_values(by=['Mean'], ascending=False)

Min        Max        Mean        Std
1 -148.330551 -30.318661  -51.261717  10.234945
7 -168.943390 -52.266983  -81.737208  14.238732
9 -169.000061 -56.443256  -88.358802  15.148681
4 -168.322922 -60.762089  -93.626212  14.114176
6 -191.840851 -58.173107  -95.923945  16.610480
3 -185.349472 -60.604523 -102.730014  16.106900
5 -178.582642 -64.516640 -103.482326  16.099102
0 -184.790421 -71.672157 -106.558980  13.476132
2 -200.037292 -63.564743 -109.273249  15.578474
8 -214.786057 -65.903961 -110.233296  18.129814

In [93]:
# Now visualise a (original/reconstruction) pair for
# each min elbo, max elbo, and approximate mean elbo
# for each label

def reconstruct(image):
    assert image.shape == (28, 28)
    out = enc_dec_model.predict_on_batch(expand_dims(image, axis=0))
    return out.reshape((28, 28))

image_pairs = []

for num in range(10):
    idx = y_df[y_df['label'] == num].index
    df_nums = df_elbos.iloc[idx, 0]
    
    # find the min/max indices
    idx_min = df_nums.idxmin()
    idx_max = df_nums.idxmax()
    
    # get the min/max (original/reconstruction) pair
    x_min_or = x_train[idx_min]
    x_min_re = reconstruct(x_min_or)
    x_min_pair = (x_min_or, x_min_re)
    
    x_max_or = x_train[idx_max]
    x_max_re = reconstruct(x_max_or)
    x_max_pair = (x_max_or, x_max_re)
    
    # to find the image with an elbo closest to the mean, we simply find
    # the image with the smallest euclidean distance from the elbo
    
    elbo_dist = (df_nums - df_elbo_labels.loc[num, 'Mean'])**2
    mean_idx = elbo_dist.idxmin()
    
    x_mean_or = x_train[mean_idx]
    x_mean_re = reconstruct(x_mean_or)
    x_mean_pair = (x_mean_or, x_mean_re)
    
    image_pairs.append([x_min_pair, x_max_pair, x_mean_pair])

In [94]:
fig, axes = subplots(10, 3, figsize=(5, 8), dpi=100)

titles = ('Min Elbo', 'Max Elbo', 'Mean Elbo')

for num in range(10):
    axes[num, 0].set_ylabel(num)
    for j, pair in enumerate(image_pairs[num]):
        cat = concatenate(pair, axis=1)
        axes[num, j].imshow(cat, cmap="Greys_r")
        
        # fix ticks/labels
        axes[0, j].set_title(titles[j])
        axes[num, j].set_xticks(())
        axes[num, j].set_yticks(())
        
# save the image
image_path = './mnist_images_elbo_%d_dim.png' %latent_dim
fig.savefig(image_path, bbox_inches='tight')